## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-21-20-52-26 +0000,nypost,"Epstein seen kissing, cuddling little girls in...",https://nypost.com/2025/12/21/us-news/epstein-...
1,2025-12-21-20-46-05 +0000,nypost,Tucker Carlson named ‘Antisemite of the Year’ ...,https://nypost.com/2025/12/21/us-news/tucker-c...
2,2025-12-21-20-44-09 +0000,startribune,Live: J.J. McCarthy out with hand injury; Viki...,https://www.startribune.com/minnesota-vikings-...
3,2025-12-21-20-41-00 +0000,wsj,Opinion | The EU Fights Carbon and Loses,https://www.wsj.com/opinion/the-eu-fights-carb...
4,2025-12-21-20-40-12 +0000,nypost,Cops discover pair of crocodiles during routin...,https://nypost.com/2025/12/21/us-news/cops-dis...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
104,trump,17
0,epstein,12
159,venezuela,10
8,new,9
156,oil,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
141,2025-12-21-01-59-17 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...,61
50,2025-12-21-17-30-34 +0000,nypost,US military targeting third tanker off coast o...,https://nypost.com/2025/12/21/world-news/us-mi...,51
9,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...,49
33,2025-12-21-18-40-44 +0000,nyt,Epstein Files Photos Disappear From Government...,https://www.nytimes.com/2025/12/20/us/politics...,48
17,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...,48


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
141,61,2025-12-21-01-59-17 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...
9,49,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...
17,48,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...
84,34,2025-12-21-11-01-08 +0000,nypost,Defiant sister of slain Bondi Beach rabbi — wh...,https://nypost.com/2025/12/21/world-news/defia...
20,30,2025-12-21-19-37-55 +0000,nypost,"Dave Chappelle hits out at Bill Maher, Israel ...",https://nypost.com/2025/12/21/us-news/dave-cha...
169,30,2025-12-20-21-46-48 +0000,nypost,Russian envoy KO’s Trump administration propos...,https://nypost.com/2025/12/20/world-news/russi...
15,29,2025-12-21-19-58-07 +0000,nypost,Bruce Blakeman reveals one issue ‘off the tabl...,https://nypost.com/2025/12/21/us-news/bruce-bl...
111,28,2025-12-21-09-52-21 +0000,nypost,"Australian PM Albanese booed, heckled as he si...",https://nypost.com/2025/12/21/world-news/austr...
70,27,2025-12-21-14-13-42 +0000,nypost,117th Newport Beach Christmas Boat Parade ligh...,https://nypost.com/2025/12/21/us-news/117th-ne...
91,27,2025-12-21-11-00-00 +0000,wsj,Cleveland Fed President Beth Hammack said she ...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
